In [4]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 

In [2]:
if(False):
    i=0
    for caption in webvtt.read('/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why_Can\'t_You_Fly%3F_deu.vtt'):
        print(caption)
        print(i)
        print('----------')
        print(caption.start) # start timestamp in text format
        print(caption.end) # end timestamp in text format
        print(caption.text) # caption text
        i=i+1

In [3]:
rootdir1 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/1_preprocessing/"
original_file_backup1 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/2_backup"




rootdir = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/test"
original_file_backup = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/backup"

In [25]:
source = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-145/12_Incredible_Facts_About_Russia_180414_mixdown_FTD12_facts_DEU.wav"
source1 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-109/0154_RLL_Tornadoes_180329_mixdown_RLL_Tornadoes_deu.wav"
source2 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-328/Do You Have Free Will_mixdown_ln_FreeWill_DEU.wav"
dest = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-145"
dest1= "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-109"
dest2 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-328"
name = "abcdef"
os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav'.format(source2,dest2,name))# (source,target,name_of_file)

0

In [7]:
#1)
"""
converting all audio to wav file with the following specification. 
    -WAVE files with 16-bit
    -16 kHz
    -mono 
"""
for Dir in natsort.natsorted(os.listdir(rootdir)):
    for subdir in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        print("----------------------"+subdir+"---------------------")
        length = len(os.listdir(rootdir+"/"+Dir+"/"+subdir))
        if(length > 2):
            print("file already converted")
        else:
            for file in os.listdir(rootdir+"/"+Dir+"/"+subdir): 
                if(file.endswith(".wav")):
                    actual_filename = file[:-4]+"_C"
                    a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+subdir+"/"+file, rootdir+"/"+Dir+"/"+subdir, actual_filename))# (source,target(half path),name_of_file)
                    #converting diretly to flac did not work. so changing the format in next step
                    print(a)
                    if(a==256):
                        print("Did not get converted")
                        
#------------------------------------------------Backing up----------------------------------------------------------

for Dir in natsort.natsorted(os.listdir(rootdir)):
    for subdir in os.listdir(rootdir+"/"+Dir):
        print("----------------------"+subdir+"---------------------")
        length = len(os.listdir(rootdir+"/"+Dir+"/"+subdir))
        if(length > 2):
            #move the file
            for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
                if(file.endswith(".wav") and file.split('_')[-1]!="C.wav"):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
        if(length == 2):
            print(subdir+ " is being removed")
            #move complete folder the the backup folder
            shutil.move(rootdir+"/"+Dir+"/"+subdir, original_file_backup) # (source,target)
            

----------------------SUB-743---------------------
0
----------------------SUB-743---------------------


In [ ]:
ffmpeg -i input.flac output.wav

ffmpeg -i DEU.wav -acodec pcm_s16le -ac 1 -ar 16000 output.flac


In [8]:
# 2)
"""
creating chunks and saving in the same folder
"""
for Dir in os.listdir(rootdir):
    for subdir in os.listdir(rootdir+"/"+Dir):
        print("1. -----------------------"+subdir)
        df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
            if(file.endswith(".vtt")):                
                for caption in webvtt.read(rootdir+"/"+Dir+"/"+subdir+"/"+file):
                    df2 = pd.DataFrame([[caption.start,caption.end,caption.text]], columns=list('ABC'))
                    df = df.append(df2)
                df = df.rename(index=str, columns={"A": "start", "B": "end","C" : "text"})
        for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
            if(file.endswith(".wav") and file.split('_')[-1]=="C.wav"):                
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/"+subdir+"/"+file, "wav")
                for i in range(0, len(df)):
                    start_time = df['start'].iloc[i]
                    end_time = df['end'].iloc[i]
                    # -----------------------------START-TIME-----------------------------------
                    s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
                    s_hours = int(s_hours)
                    s_minutes = int(s_minutes)
                    s_seconds = float(s_seconds)
                    s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
                    START = s_millisecond
                    #-------------------------------END-TIME---------------------------------------
                    e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
                    e_hours = int(e_hours)
                    e_minutes = int(e_minutes)
                    e_seconds = float(e_seconds)
                    e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
                    END = e_miliseconds
                    #--------------------------------------------------------------------------
                    chunk = myaudio[START:END]
                    chunk_name = file.split('.')[-2]+"_"+str(i)+".flac"
                    if not os.path.exists(rootdir+"/"+Dir+"/"+subdir+"/"+chunk_name):
                        print("exporting", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/"+subdir+"/"+chunk_name, format="flac")
                if not os.path.exists(original_file_backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
                    print("Backing up")
                else:
                    os.remove(rootdir+"/"+Dir+"/"+subdir+"/"+file)
                    print("File already exist. So just removing")
#----------------------------------------------txt-----------------------------------------------------------------            
        txt_name = ""        
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
            if(file.endswith(".vtt")):
                txt_name = file.split(".")[-2]
        with open(rootdir+"/"+Dir+"/"+subdir+"/"+txt_name+"_C.trans.txt", "w") as txt_file:
            i=0
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):                
                if(file.endswith(".flac")):
                    wav_file_name = file.split(".")[-2]
                    #print(df['text'].iloc[i])
                    txt_file.write(wav_file_name+" "+df['text'].iloc[i].upper()+"\n")
                    i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
            if(file.endswith(".vtt")):
                if not os.path.exists(original_file_backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
                else:
                    os.remove(rootdir+"/"+Dir+"/"+subdir+"/"+file)
                    print("File already exist. So just removing")
               
               
            

1. -----------------------SUB-743
exporting 0131_DEU_C_0.flac
exporting 0131_DEU_C_1.flac
exporting 0131_DEU_C_2.flac
exporting 0131_DEU_C_3.flac
exporting 0131_DEU_C_4.flac
exporting 0131_DEU_C_5.flac
exporting 0131_DEU_C_6.flac
exporting 0131_DEU_C_7.flac
exporting 0131_DEU_C_8.flac
exporting 0131_DEU_C_9.flac
exporting 0131_DEU_C_10.flac
exporting 0131_DEU_C_11.flac
exporting 0131_DEU_C_12.flac
exporting 0131_DEU_C_13.flac
exporting 0131_DEU_C_14.flac
exporting 0131_DEU_C_15.flac
exporting 0131_DEU_C_16.flac
exporting 0131_DEU_C_17.flac
exporting 0131_DEU_C_18.flac
exporting 0131_DEU_C_19.flac
exporting 0131_DEU_C_20.flac
exporting 0131_DEU_C_21.flac
exporting 0131_DEU_C_22.flac
exporting 0131_DEU_C_23.flac
exporting 0131_DEU_C_24.flac
exporting 0131_DEU_C_25.flac
exporting 0131_DEU_C_26.flac
exporting 0131_DEU_C_27.flac
exporting 0131_DEU_C_28.flac
exporting 0131_DEU_C_29.flac
exporting 0131_DEU_C_30.flac
Backing up


In [21]:
if(False):
    SAVE_DIR = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/"
    dirName = "Why Can't You Fly_mixdown_ln_cantFly_DEU" # direct file name as directory name
    dirPath = SAVE_DIR+"/"+dirName
    #here you put a loop to read all the wav files
    myaudio = AudioSegment.from_file("/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why Can't You Fly_mixdown_ln_cantFly_DEU.wav", "wav")
    for i in range(0, len(df)):
        if not os.path.exists(dirPath):
            os.makedirs(dirPath)    
        start_time = df['start'].iloc[i]
        end_time = df['end'].iloc[i]    
        # ------------------------------------------------------------------------
        s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
        s_hours = int(s_hours)
        s_minutes = int(s_minutes)
        s_seconds = float(s_seconds)
        s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
        START = s_millisecond
        #--------------------------------------------------------------------------
        e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
        e_hours = int(e_hours)
        e_minutes = int(e_minutes)
        e_seconds = float(e_seconds)
        e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
        END = e_miliseconds
        #--------------------------------------------------------------------------
        chunk = myaudio[START:END]    
        chunk_name =  str(i)+".wav" 
        file_destination = dirPath+"/"+chunk_name
        if not os.path.exists(file_destination):
            print("exporting", chunk_name)
            chunk.export(file_destination , format="wav")



exporting 0.wav
exporting 1.wav
exporting 2.wav
exporting 3.wav
exporting 4.wav
exporting 5.wav
exporting 6.wav
exporting 7.wav
exporting 8.wav
exporting 9.wav
exporting 10.wav
exporting 11.wav
exporting 12.wav
exporting 13.wav
exporting 14.wav
exporting 15.wav
exporting 16.wav
exporting 17.wav
exporting 18.wav
exporting 19.wav
exporting 20.wav
exporting 21.wav
exporting 22.wav
exporting 23.wav


In [12]:
#Zamia data - Preprocessing


"""
converting all audio to wav file with the following specification. 
    -flac files with 16-bit
    -16 kHz
    -mono 
"""
A = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/A"
E = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/E"
F = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/F"
G = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/G"
H = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/H"
J = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/J"
K = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/K"
M = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/M"
O = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/O"
R = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/R"
AT = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_2_Zamia/A_test"

rootdir = G




In [13]:
for Dir in natsort.natsorted(os.listdir(rootdir)):
    print(Dir)
    for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        if(file.endswith(".wav")):
            print(file)
            file_name = file[:-4]            
            a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}.wav"'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/a",file_name))
            #os.remove(rootdir+"/"+Dir+"/"+file)
            print(a)
            os.rename(rootdir+"/"+Dir+"/a/"+file,rootdir+"/"+Dir+"/a/"+file_name+".flac")
            

guenter-20160803-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
de5-004.wav
0
de5-005.wav
0
de5-006.wav
0
de5-007.wav
0
de5-008.wav
0
de5-009.wav
0
guenter-20160814-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
guenter-20160817-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
de5-004.wav
0
de5-005.wav
0
de5-006.wav
0
de5-007.wav
0
de5-008.wav
0
de5-009.wav
0
de5-010.wav
0
de5-011.wav
0
de5-012.wav
0
de5-013.wav
0
de5-014.wav
0
de5-015.wav
0
de5-016.wav
0
de5-017.wav
0
de5-018.wav
0
de5-019.wav
0
de5-020.wav
0
de5-021.wav
0
de5-022.wav
0
de5-023.wav
0
de5-024.wav
0
de5-025.wav
0
de5-026.wav
0
de5-027.wav
0
de5-028.wav
0
de5-029.wav
0
de5-030.wav
0
de5-031.wav
0
de5-032.wav
0
de5-033.wav
0
de5-034.wav
0
de5-035.wav
0
de5-036.wav
0
de5-037.wav
0
de5-038.wav
0
de5-039.wav
0
de5-040.wav
0
de5-041.wav
0
de5-042.wav
0
de5-043.wav
0
de5-044.wav
0
de5-045.wav
0
de5-046.wav
0
de5-047.wav
0
de5-048.wav
0
de5-049.wav
0
de5-050.wav
0
de5-051.wav
0
de5-052.wav
0
de5-053.wav
0
de5-054.wav
0
de5-055.wav
0

0
de5-160.wav
0
de5-161.wav
0
de5-162.wav
0
de5-163.wav
0
de5-164.wav
0
de5-165.wav
0
de5-166.wav
0
de5-167.wav
0
de5-168.wav
0
de5-169.wav
0
de5-170.wav
0
de5-171.wav
0
de5-172.wav
0
de5-173.wav
0
de5-174.wav
0
de5-175.wav
0
de5-176.wav
0
de5-177.wav
0
de5-178.wav
0
de5-179.wav
0
de5-181.wav
0
de5-182.wav
0
de5-183.wav
0
de5-184.wav
0
de5-185.wav
0
de5-186.wav
0
de5-187.wav
0
de5-188.wav
0
de5-189.wav
0
de5-190.wav
0
de5-191.wav
0
de5-192.wav
0
de5-193.wav
0
de5-194.wav
0
de5-195.wav
0
de5-196.wav
0
de5-197.wav
0
de5-198.wav
0
de5-199.wav
0
de5-200.wav
0
de5-201.wav
0
de5-202.wav
0
de5-203.wav
0
de5-204.wav
0
de5-205.wav
0
de5-206.wav
0
de5-207.wav
0
de5-208.wav
0
de5-209.wav
0
de5-210.wav
0
de5-211.wav
0
de5-212.wav
0
de5-213.wav
0
de5-214.wav
0
de5-215.wav
0
de5-216.wav
0
de5-217.wav
0
de5-218.wav
0
de5-219.wav
0
de5-220.wav
0
de5-221.wav
0
de5-222.wav
0
de5-223.wav
0
de5-224.wav
0
de5-225.wav
0
de5-226.wav
0
de5-227.wav
0
de5-228.wav
0
de5-229.wav
0
de5-230.wav
0
de5-231.wav
0
de5-

0
de5-506.wav
0
de5-507.wav
0
de5-508.wav
0
de5-509.wav
0
de5-510.wav
0
de5-511.wav
0
de5-512.wav
0
de5-513.wav
0
de5-514.wav
0
de5-515.wav
0
de5-516.wav
0
de5-517.wav
0
de5-518.wav
0
de5-519.wav
0
de5-520.wav
0
de5-521.wav
0
de5-522.wav
0
de5-523.wav
0
de5-524.wav
0
de5-525.wav
0
de5-526.wav
0
de5-527.wav
0
de5-528.wav
0
de5-529.wav
0
de5-530.wav
0
de5-531.wav
0
de5-532.wav
0
de5-533.wav
0
de5-534.wav
0
de5-535.wav
0
de5-536.wav
0
de5-537.wav
0
de5-538.wav
0
de5-539.wav
0
de5-540.wav
0
de5-541.wav
0
de5-542.wav
0
de5-543.wav
0
de5-544.wav
0
de5-545.wav
0
de5-546.wav
0
de5-547.wav
0
de5-548.wav
0
guenter-20160911-rec
de5-001.wav
0
guenter-20160913-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
de5-004.wav
0
de5-005.wav
0
de5-006.wav
0
de5-007.wav
0
de5-008.wav
0
de5-009.wav
0
de5-010.wav
0
de5-011.wav
0
de5-012.wav
0
de5-013.wav
0
de5-014.wav
0
de5-015.wav
0
de5-016.wav
0
de5-017.wav
0
de5-018.wav
0
de5-019.wav
0
de5-020.wav
0
de5-021.wav
0
de5-022.wav
0
de5-023.wav
0
de5-024.wav
0
de5-

0
de5-141.wav
0
de5-142.wav
0
de5-143.wav
0
de5-144.wav
0
de5-145.wav
0
de5-146.wav
0
de5-147.wav
0
de5-148.wav
0
de5-149.wav
0
de5-150.wav
0
de5-151.wav
0
de5-152.wav
0
de5-153.wav
0
de5-154.wav
0
de5-155.wav
0
de5-156.wav
0
de5-157.wav
0
de5-158.wav
0
de5-159.wav
0
de5-160.wav
0
de5-161.wav
0
de5-162.wav
0
de5-163.wav
0
de5-164.wav
0
de5-165.wav
0
de5-166.wav
0
de5-167.wav
0
de5-168.wav
0
de5-169.wav
0
de5-170.wav
0
de5-171.wav
0
de5-172.wav
0
de5-173.wav
0
de5-174.wav
0
de5-175.wav
0
de5-176.wav
0
de5-177.wav
0
de5-178.wav
0
de5-179.wav
0
de5-180.wav
0
de5-181.wav
0
de5-182.wav
0
de5-183.wav
0
de5-184.wav
0
de5-185.wav
0
de5-186.wav
0
de5-187.wav
0
de5-188.wav
0
de5-189.wav
0
de5-190.wav
0
de5-191.wav
0
de5-192.wav
0
de5-193.wav
0
de5-194.wav
0
de5-195.wav
0
de5-196.wav
0
de5-197.wav
0
de5-198.wav
0
de5-199.wav
0
de5-200.wav
0
de5-201.wav
0
de5-202.wav
0
de5-203.wav
0
de5-204.wav
0
de5-205.wav
0
de5-206.wav
0
de5-207.wav
0
de5-208.wav
0
de5-209.wav
0
de5-210.wav
0
de5-211.wav
0
de5-

0
de5-008.wav
0
de5-009.wav
0
de5-010.wav
0
de5-011.wav
0
de5-014.wav
0
guenter-20170925-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
de5-004.wav
0
de5-005.wav
0
de5-006.wav
0
de5-007.wav
0
de5-008.wav
0
de5-009.wav
0
de5-010.wav
0
de5-011.wav
0
de5-012.wav
0
de5-013.wav
0
de5-014.wav
0
de5-015.wav
0
de5-016.wav
0
de5-017.wav
0
de5-018.wav
0
de5-019.wav
0
de5-020.wav
0
de5-021.wav
0
de5-022.wav
0
de5-023.wav
0
de5-024.wav
0
de5-025.wav
0
de5-026.wav
0
de5-028.wav
0
de5-029.wav
0
de5-030.wav
0
de5-031.wav
0
de5-032.wav
0
de5-033.wav
0
de5-034.wav
0
de5-035.wav
0
de5-036.wav
0
de5-037.wav
0
de5-038.wav
0
de5-039.wav
0
de5-040.wav
0
de5-041.wav
0
de5-042.wav
0
de5-043.wav
0
de5-044.wav
0
de5-045.wav
0
de5-046.wav
0
de5-047.wav
0
de5-048.wav
0
de5-049.wav
0
de5-050.wav
0
de5-051.wav
0
de5-052.wav
0
de5-053.wav
0
de5-054.wav
0
de5-055.wav
0
de5-056.wav
0
de5-057.wav
0
de5-058.wav
0
de5-059.wav
0
de5-060.wav
0
de5-061.wav
0
de5-062.wav
0
de5-063.wav
0
de5-064.wav
0
de5-065.wav
0
de5-066.wav

0
de5-049.wav
0
de5-050.wav
0
de5-051.wav
0
de5-052.wav
0
de5-053.wav
0
de5-054.wav
0
de5-055.wav
0
de5-056.wav
0
de5-057.wav
0
de5-058.wav
0
de5-059.wav
0
de5-060.wav
0
de5-061.wav
0
de5-062.wav
0
de5-063.wav
0
de5-064.wav
0
de5-065.wav
0
de5-066.wav
0
guenter-20171119-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
de5-004.wav
0
de5-005.wav
0
de5-006.wav
0
de5-007.wav
0
de5-008.wav
0
de5-009.wav
0
de5-010.wav
0
de5-011.wav
0
de5-012.wav
0
de5-013.wav
0
de5-014.wav
0
de5-015.wav
0
de5-016.wav
0
de5-017.wav
0
de5-018.wav
0
de5-019.wav
0
de5-020.wav
0
de5-021.wav
0
guenter-20171127-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
guenter-20171128-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
de5-004.wav
0
de5-005.wav
0
de5-006.wav
0
de5-007.wav
0
de5-008.wav
0
guenter-20171129-rec
de5-001.wav
0
de5-002.wav
0
de5-003.wav
0
de5-004.wav
0
de5-005.wav
0
de5-006.wav
0
de5-007.wav
0
de5-008.wav
0
de5-009.wav
0
de5-010.wav
0
de5-011.wav
0
de5-012.wav
0
de5-013.wav
0
de5-014.wav
0
de5-015.wav
0
de5-

0
de5-079.wav
0
de5-080.wav
0
de5-081.wav
0
de5-082.wav
0
de5-083.wav
0
de5-084.wav
0
de5-085.wav
0
de5-086.wav
0
de5-087.wav
0
de5-088.wav
0
de5-089.wav
0
de5-090.wav
0
de5-091.wav
0
de5-092.wav
0
de5-093.wav
0
de5-094.wav
0
de5-095.wav
0
de5-096.wav
0
de5-097.wav
0
de5-098.wav
0
de5-099.wav
0
de5-100.wav
0
de5-101.wav
0
de5-102.wav
0
de5-103.wav
0
de5-104.wav
0
de5-105.wav
0
de5-106.wav
0
de5-107.wav
0
de5-108.wav
0
de5-109.wav
0
de5-110.wav
0
de5-111.wav
0
de5-112.wav
0
de5-113.wav
0
de5-114.wav
0
de5-115.wav
0
de5-116.wav
0
de5-117.wav
0
de5-118.wav
0
de5-119.wav
0
de5-120.wav
0
de5-121.wav
0
de5-122.wav
0
de5-123.wav
0
de5-124.wav
0


In [ ]:
for Dir in natsort.natsorted(os.listdir(rootdir)):
    for subdir in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        print("----------------------"+subdir+"---------------------")
        length = len(os.listdir(rootdir+"/"+Dir+"/"+subdir))
        if(length > 2):
            print("file already converted")
        else:
            for file in os.listdir(rootdir+"/"+Dir+"/"+subdir): 
                if(file.endswith(".wav")):
                    actual_filename = file[:-4]+"_C"
                    a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+subdir+"/"+file, rootdir+"/"+Dir+"/"+subdir, actual_filename))# (source,target(half path),name_of_file)
                    print(a)
                    if(a==256):
                        print("Did not get converted")

In [103]:
#-------------------------------- voxforge data prep------------------------------------------------


if False:
    #check wav file specifications
    import wave
    rootdir1 = '/home/megha/Desktop/a'

    for files in natsort.natsorted(os.listdir(rootdir1)):
        #print(files)
        f = wave.open(rootdir1+"/"+files, 'r')
        print(f.getnchannels())
        #print(f.getsampwidth())
        print(f.getframerate())



#renaming into flac file

import wave
import shutil


rootdir = '/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_3_voxforge---2'
rootdir1 = '/home/megha/Desktop/a'
for Dir in natsort.natsorted(os.listdir(rootdir)):
    for subdir in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        if(subdir == 'wav'):
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
                #print(file)
                if(file.endswith(".wav")):
                    #print('-----')
                    os.rename(rootdir+"/"+Dir+"/"+subdir+"/"+file,rootdir+"/"+Dir+"/"+subdir+"/"+file[:-4]+".flac")
                    
                    
# rename the folder name and moving primpts_original into flac folder


for Dir in natsort.natsorted(os.listdir(rootdir)):
    for subdir in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        #print(subdir)
        
        # move wav files to parent directory
        if(subdir == 'wav' or subdir == 'flac' ):
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
                #print(file)
                shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file,rootdir+"/"+Dir)                
                #move transcripts to parent directory
        if(subdir == 'etc'):
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
                #print(file)
                if(file == "prompts-original"):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file,rootdir+"/"+Dir)                    
        if(subdir == 'LICENSE'):
            os.remove(rootdir+"/"+Dir+"/"+subdir) 
            
        if(subdir == 'wav' or subdir == 'flac' or subdir == 'etc'):
            shutil.rmtree(rootdir+"/"+Dir+"/"+subdir)
            #print('deleted')  
            
# renaming the transcript file 


file_name = ''
for Dir in natsort.natsorted(os.listdir(rootdir)):
    print(Dir)
    i = 1
    for subdir in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        if(subdir.endswith(".flac")):
            while i == 1:
                file_name = subdir.split('-')[-2] 
                i = 0
        for subdir in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):  
            #print(subdir)        
            if(subdir == 'prompts-original'):
                os.rename(rootdir+"/"+Dir+"/"+subdir,rootdir+"/"+Dir+"/"+file_name+".trans.txt")

Manu-20140505-ljq
Manu-20140505-mcz
Manu-20140505-nwy
Manu-20140505-udj
Manu-20140505-uhv
Manu-20140505-wnh
Manu-20140505-wxo
Manu-20140507
Manu-20140511
Manu-20140513
Manu-20140707
Manu-20140717-iqa
Manu-20140717-wkk
Manu-20140718-adv
Manu-20140718-aen
Manu-20140718-ccs
Manu-20140718-cjp
Manu-20140718-eki
Manu-20140718-fmr
Manu-20140718-gqk
Manu-20140718-gxb
Manu-20140718-hba
Manu-20140718-hqc
Manu-20140718-hsx
Manu-20140718-ima
Manu-20140718-izs
Manu-20140718-jeu
Manu-20140718-kfk
Manu-20140718-knc
Manu-20140718-kxl
Manu-20140718-lid
Manu-20140718-mhp
Manu-20140718-mkx
Manu-20140718-moh
Manu-20140718-ntg
Manu-20140718-qos
Manu-20140718-ren
Manu-20140718-sfa
Manu-20140718-sik
Manu-20140718-sld
Manu-20140718-sob
Manu-20140718-ulw
Manu-20140718-uyb
Manu-20140718-wdg
Manu-20140718-wrr
Manu-20140718-xcg
Manu-20140718-xdh
Manu-20140718-xvu
Manu-20140718-xzd
Manu-20140718-yyz
Manu-20140718-zeh
Manu-20140718-zjv
Manu-20140718-zng
Manu-20140719-ngi
Manu-201403250-m29
Me-20090224-idf
MingTran-

thhoof-20101002-bzg
thhoof-20101002-cmu
thhoof-20101002-hjg
thhoof-20101002-hrv
thhoof-20101002-hyx
thhoof-20101002-ixv
thhoof-20101002-ksp
thhoof-20101002-lgv
thhoof-20101002-msk
thhoof-20101002-nsn
thhoof-20101002-ohn
thhoof-20101002-wyn
thhoof-20101002-xdn
thhoof-20101002-xyx
thisss-20090120-brh
thisss-20090120-vit
timiobaumann-20080418-ryd
timobaumann-20071121-ziffern1
timobaumann-20071121-ziffern2
timobaumann-20080418-bau
timobaumann-20080418-nds
timobaumann-20080418-tkh
timobaumann-20080418-tvr
timobaumann-20080418-xtb
timobaumann-20080421-mhy
timobaumann-20080421-sfu
timobaumann-20080421-uml
timobaumann-20080516-ela
timobaumann-20080516-mlf
timobaumann-20080516-nfe
timobaumann-20080516-yfl
timobaumann-20080522-gcc
timobaumann-20090804-dkr
timobaumann-20090804-vak
tolleiv-20101124-glu
tolleiv-20101124-jvw
tolleiv-20101124-nyv
tolleiv-20101124-sme
tolleiv-20101124-yzx
tolleiv-20101125-epw
tolleiv-20101125-gjs
tolleiv-20101125-jel
tolleiv-20101125-yvc
vfsh-20141230-hks
wasawasa-201

In [113]:
rootdir = '/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutsch_data/Dataset_3_voxforge---3'
for Dir in natsort.natsorted(os.listdir(rootdir)):
    print(Dir)
    for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        print(file)
    print('--------------------------------------')

1337ad-20170321-amr
de4-67.flac
de4-68.flac
de4-69.flac
de4-70.flac
de4-71.flac
de4-72.flac
de4-73.flac
de4-74.flac
de4-75.flac
de4-76.flac
de4.trans.txt
--------------------------------------
1337ad-20170321-bej
de6-47.flac
de6-48.flac
de6-49.flac
de6-50.flac
de6-51.flac
de6-52.flac
de6-53.flac
de6-54.flac
de6-55.flac
de6-56.flac
de6.trans.txt
--------------------------------------
1337ad-20170321-blf
de9-159.flac
de9-160.flac
de9-161.flac
de9-162.flac
de9-163.flac
de9-164.flac
de9-165.flac
de9-166.flac
de9-167.flac
de9-168.flac
de9.trans.txt
--------------------------------------
1337ad-20170321-czb
de11-095.flac
de11-096.flac
de11-097.flac
de11-098.flac
de11-099.flac
de11-100.flac
de11-101.flac
de11-102.flac
de11-103.flac
de11-104.flac
de11.trans.txt
--------------------------------------
1337ad-20170321-hii
de9-143.flac
de9-144.flac
de9-145.flac
de9-146.flac
de9-147.flac
de9-148.flac
de9-149.flac
de9-150.flac
de9-151.flac
de9-152.flac
de9.trans.txt
---------------------------------

de11-062.flac
de11-063.flac
de11-064.flac
de11.trans.txt
--------------------------------------
Manu-20140320-len
de2-03.flac
de2-04.flac
de2-05.flac
de2-06.flac
de2-07.flac
de2-08.flac
de2-09.flac
de2-10.flac
de2-11.flac
de2-12.flac
de2.trans.txt
--------------------------------------
Manu-20140320-mcd
de7-006.flac
de7-007.flac
de7-008.flac
de7-009.flac
de7-010.flac
de7-011.flac
de7-012.flac
de7-013.flac
de7-014.flac
de7-015.flac
de7.trans.txt
--------------------------------------
Manu-20140320-mng
de3-12.flac
de3-13.flac
de3-14.flac
de3-15.flac
de3-16.flac
de3-17.flac
de3-18.flac
de3-19.flac
de3-20.flac
de3-21.flac
de3.trans.txt
--------------------------------------
Manu-20140320-mvv
de8-118.flac
de8-119.flac
de8-120.flac
de8-121.flac
de8-122.flac
de8-123.flac
de8-124.flac
de8-125.flac
de8-126.flac
de8-127.flac
de8.trans.txt
--------------------------------------
Manu-20140320-ozr
de9-009.flac
de9-010.flac
de9-011.flac
de9-012.flac
de9-013.flac
de9-014.flac
de9-015.flac
de9-016.fla

deManu-01.flac
deManu-02.flac
deManu-03.flac
deManu-04.flac
deManu-05.flac
deManu-06.flac
deManu-07.flac
deManu-08.flac
deManu-09.flac
deManu-10.flac
deManu-11.flac
deManu-12.flac
deManu-13.flac
deManu.trans.txt
--------------------------------------
Manu-20140421-MUS
201.trans.txt
deManu-201-01.flac
deManu-201-02.flac
deManu-201-03.flac
deManu-201-04.flac
deManu-201-05.flac
deManu-201-06.flac
deManu-201-07.flac
deManu-201-08.flac
deManu-201-09.flac
deManu-201-10.flac
deManu-201-11.flac
deManu-201-12.flac
deManu-201-13.flac
deManu-201-14.flac
--------------------------------------
Manu-20140502-EM
20140502.trans.txt
deManu-20140502-01.flac
deManu-20140502-02.flac
deManu-20140502-03.flac
deManu-20140502-04.flac
deManu-20140502-05.flac
deManu-20140502-06.flac
deManu-20140502-07.flac
deManu-20140502-08.flac
deManu-20140502-09.flac
deManu-20140502-10.flac
deManu-20140502-11.flac
deManu-20140502-12.flac
deManu-20140502-13.flac
deManu-20140502-14.flac
deManu-20140502-15.flac
deManu-20140502-

de11-083.flac
de11-084.flac
de11-085.flac
de11-086.flac
de11-087.flac
de11-088.flac
de11-089.flac
de11-090.flac
de11-091.flac
de11-092.flac
de11.trans.txt
--------------------------------------
anonymous-20100119-xfm
de11-016.flac
de11-017.flac
de11-018.flac
de11-019.flac
de11-020.flac
de11-021.flac
de11-022.flac
de11-023.flac
de11-024.flac
de11-025.flac
de11.trans.txt
--------------------------------------
anonymous-20100302-huu
de8-086.flac
de8-087.flac
de8-088.flac
de8-089.flac
de8-090.flac
de8-091.flac
de8-092.flac
de8-093.flac
de8-094.flac
de8-095.flac
de8.trans.txt
--------------------------------------
anonymous-20100312-pdl
de7-086.flac
de7-087.flac
de7-088.flac
de7-089.flac
de7-090.flac
de7-091.flac
de7-092.flac
de7-093.flac
de7-094.flac
de7-095.flac
de7.trans.txt
--------------------------------------
anonymous-20100406-cxv
de8-052.flac
de8-053.flac
de8-054.flac
de8-055.flac
de8-056.flac
de8-057.flac
de8-058.flac
de8-059.flac
de8-060.flac
de8-061.flac
de8.trans.txt
----------

de5-086.flac
de5-087.flac
de5-088.flac
de5-089.flac
de5-090.flac
de5-091.flac
de5-092.flac
de5-093.flac
de5-094.flac
de5-095.flac
de5-096.flac
de5-097.flac
de5-098.flac
de5-099.flac
de5-100.flac
de5.trans.txt
--------------------------------------
guenter-20131124-ftr
de5-001.flac
de5-002.flac
de5-003.flac
de5-004.flac
de5-005.flac
de5-006.flac
de5-007.flac
de5-008.flac
de5-009.flac
de5-010.flac
de5-011.flac
de5-012.flac
de5-013.flac
de5-014.flac
de5-015.flac
de5-016.flac
de5-017.flac
de5-018.flac
de5-019.flac
de5-020.flac
de5-021.flac
de5-022.flac
de5-023.flac
de5-024.flac
de5-025.flac
de5-026.flac
de5-027.flac
de5-028.flac
de5-029.flac
de5-030.flac
de5-031.flac
de5.trans.txt
--------------------------------------
guenter-20131124-ofp
de5-001.flac
de5-002.flac
de5-003.flac
de5-004.flac
de5-005.flac
de5-006.flac
de5-007.flac
de5-008.flac
de5-009.flac
de5-010.flac
de5-011.flac
de5-012.flac
de5-013.flac
de5-014.flac
de5-015.flac
de5-016.flac
de5-017.flac
de5-018.flac
de5-019.flac
de5-020

...

de40-01.flac
de40-02.flac
de40-03.flac
de40-04.flac
de40-05.flac
de40-06.flac
de40-07.flac
de40-08.flac
de40-09.flac
de40-10.flac
de40-11.flac
de40-12.flac
de40-13.flac
de40-14.flac
de40-15.flac
de40-16.flac
de40-17.flac
de40-18.flac
de40-19.flac
de40-20.flac
de40-21.flac
de40-22.flac
de40-23.flac
de40-24.flac
de40-25.flac
de40-26.flac
de40-27.flac
de40-28.flac
de40-29.flac
de40-30.flac
de40-31.flac
de40-32.flac
de40-33.flac
de40-34.flac
de40-35.flac
de40-36.flac
de40-37.flac
de40-38.flac
de40-39.flac
de40-40.flac
de40-41.flac
de40-42.flac
de40-43.flac
de40-44.flac
de40-45.flac
de40-46.flac
de40-47.flac
de40-48.flac
de40-49.flac
de40-50.flac
de40-51.flac
de40-52.flac
de40-53.flac
de40-54.flac
de40-55.flac
de40-56.flac
de40-57.flac
de40-58.flac
de40-59.flac
de40-60.flac
de40-61.flac
de40-62.flac
de40-63.flac
de40-64.flac
de40-65.flac
de40-66.flac
de40-67.flac
de40-68.flac
de40-69.flac
de40-70.flac
de40-71.flac
de40-72.flac
de40-73.flac
de40-74.flac
de40-75.flac
de40-76.flac
de40-77.flac

de75-02.flac
de75-03.flac
de75-04.flac
de75-05.flac
de75-06.flac
de75-07.flac
de75-08.flac
de75-09.flac
de75-10.flac
de75-11.flac
de75-12.flac
de75-13.flac
de75-14.flac
de75-15.flac
de75-16.flac
de75-17.flac
de75-18.flac
de75-19.flac
de75-20.flac
de75-21.flac
de75-22.flac
de75-23.flac
de75-24.flac
de75-25.flac
de75-26.flac
de75-27.flac
de75-28.flac
de75-29.flac
de75-30.flac
de75-31.flac
de75-32.flac
de75-33.flac
de75-34.flac
de75-35.flac
de75-36.flac
de75-37.flac
de75-38.flac
de75-39.flac
de75-40.flac
de75-41.flac
de75-42.flac
de75-43.flac
de75-44.flac
de75-45.flac
de75-46.flac
de75-47.flac
de75-48.flac
de75-49.flac
de75-50.flac
de75-51.flac
de75-52.flac
de75-53.flac
de75-54.flac
de75-55.flac
de75-56.flac
de75-57.flac
de75-58.flac
de75-59.flac
de75-60.flac
de75-61.flac
de75-62.flac
de75-63.flac
de75-64.flac
de75-65.flac
de75-66.flac
de75-67.flac
de75-68.flac
de75-69.flac
de75-70.flac
de75-71.flac
de75-72.flac
de75-73.flac
de75-74.flac
de75-75.flac
de75-76.flac
de75-77.flac
de75-78.flac

de109-01.flac
de109-02.flac
de109-03.flac
de109-04.flac
de109-05.flac
de109-06.flac
de109-07.flac
de109-08.flac
de109-09.flac
de109-10.flac
de109-11.flac
de109-12.flac
de109-13.flac
de109-14.flac
de109-15.flac
de109-16.flac
de109-17.flac
de109-18.flac
de109-19.flac
de109-20.flac
de109-21.flac
de109-22.flac
de109-23.flac
de109-24.flac
de109-25.flac
de109-26.flac
de109-27.flac
de109-28.flac
de109-29.flac
de109-30.flac
de109-31.flac
de109-32.flac
de109-33.flac
de109-34.flac
de109-35.flac
de109-36.flac
de109-37.flac
de109-38.flac
de109-39.flac
de109-40.flac
de109-41.flac
de109-42.flac
de109-43.flac
de109-44.flac
de109-45.flac
de109-46.flac
de109-47.flac
de109-48.flac
de109-49.flac
de109-50.flac
de109-51.flac
de109-52.flac
de109-53.flac
de109-54.flac
de109-55.flac
de109-56.flac
de109-57.flac
de109-58.flac
de109-59.flac
de109-60.flac
de109-61.flac
de109-62.flac
de109-63.flac
de109-64.flac
de109-65.flac
de109-66.flac
de109-67.flac
de109-68.flac
de109-69.flac
de109-70.flac
de109-71.flac
de109-

de138-01.flac
de138-02.flac
de138-03.flac
de138-04.flac
de138-05.flac
de138-06.flac
de138-07.flac
de138-08.flac
de138-09.flac
de138-10.flac
de138-11.flac
de138-12.flac
de138-13.flac
de138-14.flac
de138-15.flac
de138-16.flac
de138-17.flac
de138-18.flac
de138-19.flac
de138-20.flac
de138-21.flac
de138-22.flac
de138-23.flac
de138-24.flac
de138-25.flac
de138-26.flac
de138-27.flac
de138-28.flac
de138-29.flac
de138-30.flac
de138-31.flac
de138-32.flac
de138-33.flac
de138-34.flac
de138-35.flac
de138-36.flac
de138-37.flac
de138-38.flac
de138-39.flac
de138-40.flac
de138-41.flac
de138-42.flac
de138-43.flac
de138-44.flac
de138-45.flac
de138-46.flac
de138-47.flac
de138-48.flac
de138-49.flac
de138-50.flac
de138-51.flac
de138-52.flac
de138-53.flac
de138-54.flac
de138-55.flac
de138-56.flac
de138-57.flac
de138-58.flac
de138-59.flac
de138-60.flac
de138-61.flac
de138-62.flac
de138-63.flac
de138-64.flac
de138-65.flac
de138-66.flac
de138-67.flac
de138-68.flac
de138-69.flac
de138-70.flac
de138-71.flac
de138-

de4-88.flac
de4-89.flac
de4-90.flac
de4-91.flac
de4-92.flac
de4-93.flac
de4-94.flac
de4-95.flac
de4-96.flac
de4.trans.txt
--------------------------------------
timobaumann-20080516-ela
de11-100.flac
de11-101.flac
de11-102.flac
de11-103.flac
de11-104.flac
de11-105.flac
de11-106.flac
de11-107.flac
de11-108.flac
de11-109.flac
de11.trans.txt
--------------------------------------
timobaumann-20080516-mlf
de9-055.flac
de9-056.flac
de9-057.flac
de9-058.flac
de9-059.flac
de9-060.flac
de9-061.flac
de9-062.flac
de9-063.flac
de9-064.flac
de9.trans.txt
--------------------------------------
timobaumann-20080516-nfe
de5-015.flac
de5-016.flac
de5-017.flac
de5-018.flac
de5-019.flac
de5-020.flac
de5-021.flac
de5-022.flac
de5-023.flac
de5-024.flac
de5.trans.txt
--------------------------------------
timobaumann-20080516-yfl
de10-045.flac
de10-046.flac
de10-047.flac
de10-048.flac
de10-049.flac
de10-050.flac
de10-051.flac
de10-052.flac
de10-053.flac
de10-054.flac
de10.trans.txt
------------------------

In [ ]:
Manu-20100324-m25 
deM25
Manu-20130329-m43
deM43
Manu-20140216-m1
de_M1
Manu-20140323-m2
deM2
Manu-20140323-m3
deM3
Manu-20140323-m4
deM4
Manu-20140323-m5
deM5
Manu-20140323-m6
deM6
Manu-20140323-m7
deM7
Manu-20140323-m8
deM8
Manu-20140323-m9
deM9
Manu-20140323-m10
deM10
Manu-20140323-m11
deM11
Manu-20140323-m12
deM12
Manu-20140324-m13
deM13
Manu-20140324-m14
deM14
Manu-20140324-m15
deM15
Manu-20140324-m16
deM16    ----------------till 51

Manu-20140328-cc1
deCC1
deCC2
deCC3
deCC4

Manu-20140419
deManu

